# LLM Optimization Modelling Experiment

In [1]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Image
from IPython.display import Markdown

## 1. Define the problem description

In [553]:
problem = '''You are a chemist in charge of a process that requires two liquids to reach the end product. Your goal is to maximize the quality of the product in the end, which is measured by the difference between the Napierian logarithm of the product of the litres used of each component and the cube root of the summation of the first component litres as well as double the second component litres. It is required that the squared difference between the litres employed of each component and the mean litres utilised of both components is below ten. This is necessary for the obtained set to be stable. Furthermore, the mixture tank available to you can only fit 80 l. Please formulate a mathematical optimization model for this problem. '''

## 2. Ask for parameters

In [554]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Please formulate only the variables for this mathematical optimization problem. 
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [555]:
#Show the resopnse in a formatted way
Markdown(response.text)

### Variables:

*  Let  'x' be the litres used of the first liquid.
*  Let 'y' be the litres used of the second liquid. 


# 2. Ask for objective

In [556]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please formulate only the objective function for this mathematical optimization problem."
prompt2 += problem + response.text
response2 = generative_multimodal_model.generate_content([prompt2])

In [557]:
Markdown(response2.text)

## Objective function: 

Maximize:  ln(x*y) - (x + 2y)^(1/3) 


# 3. Ask for constraints

In [558]:
#Second prompt gets the output of the previous step and generates the code
prompt3 = "Please formulate only the constraints for this mathematical optimization problem."
prompt3 += problem + response.text + response2.text
response3 = generative_multimodal_model.generate_content([prompt3])

In [559]:
Markdown(response3.text)

## Constraints:

1. **Tank capacity:** 
   *  x + y <= 80 

2. **Stability requirement:**
   * (x - (x+y)/2)^2  <= 10
   * (y - (x+y)/2)^2  <= 10

3. **Non-negativity of variables:**
   * x >= 0
   * y >= 0 


## 3. Generate the pyomo code

In [560]:
#Second prompt gets the output of the previous step and generates the code
prompt4 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt4 += response.text + response2.text + response3.text 
response4 = generative_multimodal_model.generate_content([prompt4])

In [561]:
## Showing the code in a formatted way
Markdown(response4.text)

```python
from pyomo.environ import *

# Create a concrete model
model = ConcreteModel()

# Define variables
model.x = Var(domain=NonNegativeReals)
model.y = Var(domain=NonNegativeReals)

# Define objective function
model.obj = Objective(expr=log(model.x * model.y) - (model.x + 2 * model.y)**(1/3), sense=maximize)

# Define constraints
model.tank_capacity = Constraint(expr=model.x + model.y <= 80)
model.stability1 = Constraint(expr=(model.x - (model.x + model.y) / 2)**2 <= 10)
model.stability2 = Constraint(expr=(model.y - (model.x + model.y) / 2)**2 <= 10)

# Solve the model
solver = SolverFactory('ipopt')  # You can choose a different solver if needed
results = solver.solve(model)

# Print the results
print("Status:", results.solver.status)
print("Termination condition:", results.solver.termination_condition)
print("Optimal solution:")
print("x =", model.x.value)
print("y =", model.y.value)
print("Objective function value:", model.obj.value) 
```
 
This code defines the variables, objective function, and constraints based on your provided mathematical formulation. It then uses the 'ipopt' solver to find the optimal solution. 

**Note:** I have used the 'ipopt' solver as an example. You may need to install it separately if you don't have it already. You can also use other non-linear solvers supported by Pyomo. You can find more information about installing solvers and Pyomo here:  https://pyomo.readthedocs.io/en/stable/ 


## 4. Input problem data and try running the generated code

In [563]:
from pyomo.environ import *

# Create a concrete model
model = ConcreteModel()

# Define variables
model.x = Var(domain=NonNegativeReals)
model.y = Var(domain=NonNegativeReals)

# Define objective function
model.obj = Objective(expr=log(model.x * model.y) - (model.x + 2 * model.y)**(1/3), sense=maximize)

# Define constraints
model.tank_capacity = Constraint(expr=model.x + model.y <= 80)
model.stability1 = Constraint(expr=(model.x - (model.x + model.y) / 2)**2 <= 10)
model.stability2 = Constraint(expr=(model.y - (model.x + model.y) / 2)**2 <= 10)

# Solve the model
solver = SolverFactory('ipopt')  # You can choose a different solver if needed
results = solver.solve(model)

# Print the results
print("Status:", results.solver.status)
print("Termination condition:", results.solver.termination_condition)
print("Optimal solution:")
print("x =", model.x.value)
print("y =", model.y.value)
print("Objective function value:", model.obj()) 

Status: ok
Termination condition: optimal
Optimal solution:
x = 43.162277233647174
y = 36.8377229631074
Objective function value: 2.4827783318454744


## 5. Correct the code to verify model viability (optional)

## 6. Print the responses

In [564]:
print(response.text)

### Variables:

*  Let  'x' be the litres used of the first liquid.
*  Let 'y' be the litres used of the second liquid. 



In [565]:
print(response2.text)

## Objective function: 

Maximize:  ln(x*y) - (x + 2y)^(1/3) 



In [566]:
print(response3.text)

## Constraints:

1. **Tank capacity:** 
   *  x + y <= 80 

2. **Stability requirement:**
   * (x - (x+y)/2)^2  <= 10
   * (y - (x+y)/2)^2  <= 10

3. **Non-negativity of variables:**
   * x >= 0
   * y >= 0 



In [567]:
print(response4.text)

```python
from pyomo.environ import *

# Create a concrete model
model = ConcreteModel()

# Define variables
model.x = Var(domain=NonNegativeReals)
model.y = Var(domain=NonNegativeReals)

# Define objective function
model.obj = Objective(expr=log(model.x * model.y) - (model.x + 2 * model.y)**(1/3), sense=maximize)

# Define constraints
model.tank_capacity = Constraint(expr=model.x + model.y <= 80)
model.stability1 = Constraint(expr=(model.x - (model.x + model.y) / 2)**2 <= 10)
model.stability2 = Constraint(expr=(model.y - (model.x + model.y) / 2)**2 <= 10)

# Solve the model
solver = SolverFactory('ipopt')  # You can choose a different solver if needed
results = solver.solve(model)

# Print the results
print("Status:", results.solver.status)
print("Termination condition:", results.solver.termination_condition)
print("Optimal solution:")
print("x =", model.x.value)
print("y =", model.y.value)
print("Objective function value:", model.obj.value) 
```
 
This code defines the variables, obj